# Model
* load ternaus-net model (or my model for bone seg)
* set frozen pretrained VGG weights
* keras to estimator
* T2T keras model - TODO
* train the rest
* fine tune the whole network - TODO

References:
* [VGG16 TernausNet](https://github.com/ternaus/TernausNet/blob/master/unet_models.py) 
* [Keras Transfer Learning I](https://www.learnopencv.com/keras-tutorial-fine-tuning-using-pre-trained-models)
* [Keras Transfer Learning II](https://flyyufelix.github.io/2016/10/08/fine-tuning-in-keras-part2.html)

In [17]:
import sys
sys.path.append('../sentinel/')
sys.path.append('../')

from importlib import reload

import numpy as np
import tensorflow as tf

import vgg_unet
reload(vgg_unet)
from vgg_unet import vgg16_unet, jaccard_coef_loss, jaccard_coef_int

%matplotlib inline
%load_ext autoreload
%autoreload 2

hparams = tf.contrib.training.HParams(
    image_size=224,
    num_classes=1,
    num_filters=32,
    is_deconv=False,
    use_bn_elu=False,
    freeze_encoder=True,
)

ternaus_net = vgg16_unet(hparams)
Nadam = tf.keras.optimizers.Nadam
ternaus_net.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,
                    metrics=['binary_crossentropy', jaccard_coef_int])

estimator = tf.keras.estimator.model_to_estimator(ternaus_net)
estimator

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmplcozb2dq', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f58bb6fcb38>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# Fix the variables in the TEST

In [16]:
# ternaus_net._make_train_function()
# train_op = ternaus_net.train_function.updates_op

# var_name = 'block'
# variables = [x for x in tf.trainable_variables() if var_name in x.name]
# var_list = list(set(tf.trainable_variables()) - set(variables))

# import numpy as np
# import mltest
# feed_dict = {
#         'input_1:0': np.random.normal(size=(10,)+input_shape),
#         'logit_target:0': np.random.randint(1, size=(10, 224, 224, 1)),
#         'logit_sample_weights:0': [0.] * 10
#     }

# # Run the test suite!
# mltest.test_suite(
#     ternaus_net.output,
#     train_op,
#     feed_dict=feed_dict,
#     var_list=var_list)